# __DS 과제5__

### __3. 생일이 느린 순서로 10명의 친구 출력__

In [1]:
class Heap:
	def __init__(self, *args):
		if len(args) != 0:
			self.__A = args[0] # 파라미터로 온 리스트
		else:
			self.__A = []

	# [알고리즘 8-2] 구현: 힙에 원소 삽입하기(재귀 알고리즘 버전)
	def insert(self, x):
		self.__A.append(x)
		self.__percolateUp(len(self.__A)-1)

	# 스며오르기
	def __percolateUp(self, i:int):
		parent = (i - 1) // 2
		if i > 0 and self.__A[i] > self.__A[parent]:
			self.__A[i], self.__A[parent] = self.__A[parent], self.__A[i]
			self.__percolateUp(parent)

	# [알고리즘 8-2] 구현: 힙에서 원소 삭제하기
	def deleteMax(self):
		# heap is in self.__A[0...len(self.__A)-1]
		if (not self.isEmpty()):
			max = self.__A[0]
			self.__A[0] = self.__A.pop() # *.pop(): 리스트의 끝원소 삭제 후 원소 리턴
			self.__percolateDown(0)
			return max
		else:
			return None

	# 스며내리기
	def __percolateDown(self, i:int):
		# Percolate down w/ self.__A[i] as the root
		child = 2 * i + 1  # left child
		right = 2 * i + 2  # right child
		if (child <= len(self.__A)-1):
			if (right <= len(self.__A)-1 and self.__A[child] < self.__A[right]):
				child = right  # index of larger child
			if self.__A[i] < self.__A[child]:
				self.__A[i], self.__A[child] = self.__A[child], self.__A[i]
				self.__percolateDown(child)

	def max(self):
		return self.__A[0]

	# 힙 만들기
	def buildHeap(self):
		for i in range((len(self.__A) - 2) // 2, -1, -1):
			self.__percolateDown(i)

	# 힙이 비었는지 확인하기
	def isEmpty(self) -> bool:
		return len(self.__A) == 0

	def clear(self):
		self.__A = []

	def size(self) -> int:
		return len(self.__A)

In [4]:
# 2. 생일 데이터 파일 직접 열기
import csv
from datetime import datetime

heap = Heap()

with open("birthday.csv", encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # 헤더 건너뛰기

    for row in reader:
        try:
            name = row[1].strip()  # 이름
            birth_raw = row[2].strip()  # 생년월일8자리
            birth_date = datetime.strptime(birth_raw, "%Y%m%d")
            timestamp = birth_date.timestamp()
            heap.insert((timestamp, name, birth_raw))  # 힙에 (우선순위, 이름, 생일) 삽입
        except:
            continue  # 생일 형식 이상한 건 생략

# 3. 생일이 가장 늦은 친구 10명 출력
print("🎂 생일이 가장 늦은 친구 10명:")
for _ in range(10):
    if not heap.isEmpty():
        _, name, birth = heap.deleteMax()
        print(f"{name} - {birth}")

🎂 생일이 가장 늦은 친구 10명:
신수민 - 20051230
이서영 - 20051225
강민주 - 20051214
김민경 - 20051202
이서영 - 20051112
배시은 - 20051102
김여원 - 20051031
이서진 - 20051028
서홍빈 - 20051024
김예빈 - 20051019


### __4. 같은 조 친구들의 이름과 생년월일만 출력__

In [7]:
# 노드 클래스 정의
class BidirectNode:
    def __init__(self, item, prev, next):
        self.item = item
        self.prev = prev
        self.next = next

# 양방향 원형 연결 리스트 클래스 정의
class CircularDoublyLinkedList:
    def __init__(self):
        self.__head = BidirectNode("dummy", None, None)
        self.__head.prev = self.__head
        self.__head.next = self.__head
        self.__numItems = 0

    def insert(self, i: int, newItem) -> None:
        if 0 <= i <= self.__numItems:
            prev = self.getNode(i - 1)
            newNode = BidirectNode(newItem, prev, prev.next)
            newNode.next.prev = newNode
            prev.next = newNode
            self.__numItems += 1
        else:
            print("index", i, ": out of bound in insert()")

    def append(self, newItem) -> None:
        prev = self.__head.prev
        newNode = BidirectNode(newItem, prev, self.__head)
        prev.next = newNode
        self.__head.prev = newNode
        self.__numItems += 1

    def pop(self, *args):
        if self.isEmpty():
            return None
        if len(args) != 0:
            i = args[0]
        if len(args) == 0 or i == -1:
            i = self.__numItems - 1
        if 0 <= i <= self.__numItems - 1:
            curr = self.getNode(i)
            retItem = curr.item
            curr.prev.next = curr.next
            curr.next.prev = curr.prev
            self.__numItems -= 1
            return retItem
        else:
            return None

    def remove(self, x):
        curr = self.__findNode(x)
        if curr is not None:
            curr.prev.next = curr.next
            curr.next.prev = curr.prev
            self.__numItems -= 1
            return x
        else:
            return None

    def get(self, *args):
        if self.isEmpty():
            return None
        if len(args) != 0:
            i = args[0]
        if len(args) == 0 or i == -1:
            i = self.__numItems - 1
        if 0 <= i <= self.__numItems - 1:
            return self.getNode(i).item
        else:
            return None

    def index(self, x) -> int:
        cnt = 0
        for element in self:
            if element == x:
                return cnt
            cnt += 1
        return -12345

    def isEmpty(self) -> bool:
        return self.__numItems == 0

    def size(self) -> int:
        return self.__numItems

    def clear(self):
        self.__head = BidirectNode("dummy", None, None)
        self.__head.prev = self.__head
        self.__head.next = self.__head
        self.__numItems = 0

    def count(self, x) -> int:
        cnt = 0
        for element in self:
            if element == x:
                cnt += 1
        return cnt

    def extend(self, a):
        for element in a:
            self.append(element)

    def copy(self) -> 'CircularDoublyLinkedList':
        a = CircularDoublyLinkedList()
        for element in self:
            a.append(element)
        return a

    def reverse(self) -> None:
        prev = self.__head
        curr = prev.next
        next = curr.next
        self.__head.next = prev.prev
        self.__head.prev = curr
        for i in range(self.__numItems):
            curr.next = prev
            curr.prev = next
            prev = curr
            curr = next
            next = next.next

    def sort(self) -> None:
        a = []
        for element in self:
            a.append(element)
        a.sort()
        self.clear()
        for element in a:
            self.append(element)

    def __findNode(self, x) -> BidirectNode:
        curr = self.__head.next
        while curr != self.__head:
            if curr.item == x:
                return curr
            curr = curr.next
        return None

    def getNode(self, i: int) -> BidirectNode:
        curr = self.__head
        for index in range(i + 1):
            curr = curr.next
        return curr

    def printList(self) -> None:
        for element in self:
            print(element, end=' ')
        print()

    def __iter__(self):  # ✅ 올바른 위치에 정의
        return CircularDoublyLinkedListIterator(self)


class CircularDoublyLinkedListIterator:
    def __init__(self, alist):
        self.__head = alist.getNode(-1)  # 더미 헤드
        self.iterPosition = self.__head.next

    def __next__(self):
        if self.iterPosition == self.__head:
            raise StopIteration
        else:
            item = self.iterPosition.item
            self.iterPosition = self.iterPosition.next
            return item


In [8]:
import csv

cdll = CircularDoublyLinkedList()

with open("birthday.csv", encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # 첫 줄은 헤더이므로 건너뜀

    for row in reader:
        try:
            name = row[1].strip()
            birth = row[2].strip()
            cdll.append((name, birth))  # 리스트에 (이름, 생일) 형태로 저장
        except:
            continue

# 출력 대상 이름 리스트
target_names = [
    "권보은", "김승연", "이서영", "이아현", "임성민",
    "은유빈", "이예은", "정예은", "김주원", "서홍빈"
]

# 리스트에서 대상 학생 정보 찾기
print("🎉 선택된 학생 10명의 생일 정보:")
for item in cdll:
    name, birth = item
    if name in target_names:
        print(f"{name} - {birth}")

🎉 선택된 학생 10명의 생일 정보:
권보은 - 20041004
김승연 - 20030124
김주원 - 20030110
서홍빈 - 20051024
은유빈 - 20040503
이서영 - 20051112
이서영 - 20051225
이아현 - 20010904
이예은 - 20030109
임성민 - 20021213
정예은 - 


### __5. 교재 8장 우선 순위 큐 연습문제__

### __5-1.__

최대 힙은 완전 이진 트리의 형태를 가지며, 모든 부모 노드는 자식 노드보다 크거나 같은 값을 가져야 한다. 이 규칙은 트리 전체에 재귀적으로 적용되기 때문에, 트리에서 깊이가 얕은(루트에 가까운) 노드는 항상 깊이가 더 깊은(말단에 가까운) 노드보다 값이 크거나 같아야 한다. 따라서 더 얕은 곳에 있는 노드가 더 깊은 곳에 있는 노드보다 작은 값을 갖는 경우는 발생할 수 없다.

결론적으로, 최대 힙에서는 더 얕은 위치에 있는 원소가 더 깊은 위치에 있는 원소보다 작은 값을 가질 수 없다.

### __5-2.__

최대 힙에서 A[0]은 항상 가장 큰 값을 가진다. 이는 최대 힙의 정의에 따라 루트 노드가 전체에서 가장 큰 원소이기 때문이다. 하지만 A[n−1], 즉 배열의 마지막 위치에 있는 원소는 단지 트리를 배열에 저장할 때의 위치일 뿐이며, 값의 크기와는 무관하다. 힙은 완전 이진 트리이므로 배열로 저장할 때 트리 구조의 깊은 말단 노드들이 뒤에 위치하게 되며, 이들은 반드시 가장 작은 값을 갖는다고 보장할 수 없다.

결론적으로, A[0]은 항상 최대값이지만, A[n−1]이 항상 최소값이라는 보장은 없다.
A[n−1]은 단지 배열상 마지막 위치에 존재하는 원소일 뿐이며, 그 값은 힙 속성과는 무관하다.

### __5-3.__

- 최대 힙을 배열로 표현한 구조 A[0...n−1]에 대해 buildHeap() 알고리즘은
배열의 중간 인덱스부터 **루트(인덱스 0)**까지 거꾸로 순회하며 각 노드에 대해 percolateDown()을 수행한다.
- 배열의 중간 인덱스인 ⌊n/2⌋부터 끝까지의 노드들은 모두 **리프 노드(자식이 없는 노드)**이므로 이 노드들은 아래로 스며내릴 필요가 없다. 따라서 percolateDown()을 수행할 필요가 없는 노드는 정확히 ⌊n/2⌋개이다.

- 이 값은 힙의 전체 노드 중 **하위 절반(리프)**에 해당하며, buildHeap() 알고리즘은 이들을 제외한 나머지 노드에서만 작업을 수행한다.

결론적으로, buildHeap()에서 percolateDown을 수행하지 않고 넘어가는 노드는 정확히 ⌊n/2⌋개이다.

### __5-4.__

- 최악의 경우  
  - 각 내부 노드가 자신의 자식 노드와 비교해 아래로 끝까지 내려가야 하는 경우

  - 가장 높은 루트 노드는 log n만큼 스며내려야 함

  - 이런 경우가 전체에 걸쳐 발생한다고 가정할 때, 단순히 생각하면 O(n log n)처럼 보일 수 있음

  - 하지만 실제로는 트리의 구조상 높은 위치일수록 노드 수가 적기 때문에 전체 합은 O(n)

  **결론**: buildHeap()의 최악의 경우 시간 복잡도는 O(n)

-  최선의 경우
  - 각 노드가 한 번도 자식과 교환하지 않고 한 번의 비교만으로 힙 조건을 만족하는 경우

  - 즉, 이미 배열이 거의 힙 구조에 가까워서 추가 작업이 거의 필요 없는 상황

  - 이 경우 각 노드는 비교는 하되 교환 없이 종료되므로, 전체적으로 비교 횟수는 최소

  **결론**: 최선의 경우에도 전체 노드를 순회하므로 시간 복잡도는 여전히 O(n)

### __5-5.__

힙은 완전 이진 트리 구조를 배열로 표현한 자료구조이기 때문에,
힙의 마지막 원소는 배열의 마지막 요소 A[n−1]와 정확히 일치함.  
힙에서 마지막 원소(A[n−1])를 제거하는 것은 매우 간단하며, 별도의 힙 유지 작업 없이 O(1) 시간에 수행 가능함.

### __5-6.__

**방식 1**: 스며내리기 (percolateDown) 방식 — 본문 방식
- 인덱스 ⌊n/2⌋부터 0까지 거꾸로 순회하며 percolateDown() 수행

- 각 노드의 높이에 따라 작업 횟수가 다르고, 자식부터 정렬되므로 효율적

- 시간 복잡도는 O(n)
→ 증명된 결과로, percolateDown 방식의 buildHeap()은 선형 시간에 가능

**방식 2**: 스며오르기 (percolateUp) 방식
- 배열 A[0...n−1]의 각 원소를 앞에서부터 차례대로 힙에 삽입한다고 생각

- 삽입 시마다 스며오르기 수행 → 각 삽입 연산당 O(log n) 시간

- 총 n개 원소를 삽입해야 하므로 O(n log n) 시간 소요

**결론**: 본문에 제시한 방법에 비해 비효율적이다

### __5-7.__

힙에서 어떤 원소의 값이 증가하면, 해당 노드는 스며오르기를 통해 루트 방향으로 이동하며 힙 속성을 복구해야 한다.

### __6. LeetCode 703.Kth Largest Element in Stream__

In [12]:
import heapq

class KthLargest:

    def __init__(self, k: int, nums: list[int]):
        self.k = k
        self.min_heap = []
        for num in nums:
            self.add(num)  # 초기값도 힙에 넣기

    def add(self, val: int) -> int:
        heapq.heappush(self.min_heap, val)
        if len(self.min_heap) > self.k:
            heapq.heappop(self.min_heap)
        return self.min_heap[0]  # k번째로 큰 값

# 예시 실행
k = 3
nums = [4, 5, 8, 2]
kthLargest = KthLargest(k, nums)

# 테스트 입력
inputs = [3, 5, 10, 9, 4]

print(f"초기값: {nums} / k = {k}")
for val in inputs:
    result = kthLargest.add(val)
    print(f"add({val}) → {result}")


초기값: [4, 5, 8, 2] / k = 3
add(3) → 4
add(5) → 5
add(10) → 5
add(9) → 8
add(4) → 8
